In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/datasets

Mounted at /content/drive
/content/drive/MyDrive/datasets


In [4]:
!pip install sentence_transformers sklearn

     |████████████████████████████████| 79 kB 3.4 MB/s 
     |████████████████████████████████| 4.0 MB 9.2 MB/s 
     |████████████████████████████████| 1.2 MB 37.2 MB/s 
     |████████████████████████████████| 77 kB 4.1 MB/s 
     |████████████████████████████████| 895 kB 34.9 MB/s 
     |████████████████████████████████| 596 kB 8.9 MB/s 
     |████████████████████████████████| 6.6 MB 33.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=75b9b1d5e344b75d737a7735051d85fd11792b10cb36493545bea2fdd7f84ab6
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import numpy as np
import pandas as pd
import json
import re
import csv
pd.set_option('display.max_colwidth', 300)

data = []

with open("booksummaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in reader:
        data.append(row)

from sentence_transformers import SentenceTransformer
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

16559it [00:00, 24824.16it/s]


In [6]:
book_index = []
book_id = []
book_author = []
book_name = []
summary = []
genre = []
a = 1
for i in tqdm(data):
    book_index.append(a)
    a += 1
    book_id.append(i[0])
    book_name.append(i[2])
    book_author.append(i[3])
    genre.append(i[5])
    summary.append(i[6])

df = pd.DataFrame({'index': book_index, 'id': book_id, 'title': book_name, 'author': book_author, 'genre': genre, 'summary': summary})

100%|██████████| 16559/16559 [00:00<00:00, 155048.78it/s]


In [7]:
df.head()

,index,id,title,author,genre,summary
0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p..."
1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and..."
2,3,986,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fiction"", ""/m/0pym5"": ""Absurdist fiction"", ""/m/05hgj"": ""Novel""}","The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t..."
3,4,1756,An Enquiry Concerning Human Understanding,David Hume,,"The argument of the Enquiry proceeds by a series of incremental steps, separated into chapters which logically succeed one another. After expounding his epistemology, Hume explains how to apply his principles to specific topics. In the first section of the Enquiry, Hume provides a rough introdu..."
4,5,2080,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""Speculative fiction"", ""/m/01hmnh"": ""Fantasy"", ""/m/02xlf"": ""Fiction""}","The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti..."


In [8]:
df = df.drop(df[df['genre'] == ''].index)
df = df.drop(df[df['summary'] == ''].index)

genres_cleaned = []
for i in df['genre']:
    genres_cleaned.append(list(json.loads(i).values()))
df['genre'] = genres_cleaned
df['genrestring'] = df['genre'].apply(lambda x: ' '.join(x))

In [9]:
text_df = df[['title', 'author', 'genrestring', 'summary']]
text_df.head()

,title,author,genrestring,summary
0,Animal Farm,George Orwell,Roman à clef Satire Children's literature Speculative fiction Fiction,"Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p..."
1,A Clockwork Orange,Anthony Burgess,Science Fiction Novella Speculative fiction Utopian and dystopian fiction Satire Fiction,"Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and..."
2,The Plague,Albert Camus,Existentialism Fiction Absurdist fiction Novel,"The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t..."
4,A Fire Upon the Deep,Vernor Vinge,Hard science fiction Science Fiction Speculative fiction Fantasy Fiction,"The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti..."
5,All Quiet on the Western Front,Erich Maria Remarque,War novel Roman à clef,"The book tells the story of Paul Bäumer, a German soldier who—urged on by his school teacher—joins the German army shortly after the start of World War I. Bäumer arrives at the Western Front with his friends and schoolmates (Tjaden, Müller, Kropp and a number of other characters). There they me..."


In [10]:
summaries = list(text_df['summary'])

In [11]:
# vectorize text
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(summaries)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [12]:
cos_sim_data = pd.DataFrame(cosine_similarity(embeddings))

def recommendation_results(df, book_name):
    book_idx = df.index[df['title'] == book_name].tolist()[0]
    
    recomm_idx = cos_sim_data.loc[book_idx].sort_values(ascending=False).index.tolist()[1:4]
    recomm_books = df['title'].iloc[recomm_idx].values
    
    for i, r in enumerate(recomm_books):
        print('The #', i+1 , 'recommendation is', r)
    return recomm_books

In [15]:
recommendation_results(text_df,"Rendezvous with Rama")

The # 1 recommendation is Encounter With Tiber
The # 2 recommendation is Camouflage
The # 3 recommendation is Assemblers of Infinity


array(['Encounter With Tiber', 'Camouflage', 'Assemblers of Infinity'],
      dtype=object)